In [1]:
from magpie import grasp as gt
import rtde_control
import rtde_receive
from magpie.motor_code import Motors
from magpie import ur5 as ur5
import time
import numpy as np
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import importlib
importlib.reload(gt)
importlib.reload(ur5)

<module 'magpie.ur5' from '/home/will/MAGPIE/magpie/ur5.py'>

In [3]:
# tmat = np.array([[ 0.91574736, -0.35602909,  0.18614527, -0.02848649],
#        [-0.40073104, -0.77640133,  0.48643151,  0.14332736],
#        [ 0.02866033,  0.52004256,  0.85365937,  0.33750796],
#        [ 0.        ,  0.        ,  0.        ,  1.        ]])

tmat = np.array([[ 0.94506831, -0.26542847, -0.19077109,  0.04587254],
       [-0.31977141, -0.87173858, -0.37123859,  0.03873094],
       [-0.06776523,  0.41184897, -0.90872895,  0.23995551],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

mmc = tmat[:3, 3]
# grasp_pose = [mmc[1], -mmc[0], mmc[2]]
grasp_pose = mmc
tmat[:3, 3] = [0, 0, 0]

In [6]:
# Full test
sleepRate = 1.5
try:
    robotIP = "192.168.0.6"
    # con = rtde_control.RTDEControlInterface(robotIP)
    # rec = rtde_receive.RTDEReceiveInterface(robotIP)
    ur = ur5.UR5_Interface()
    servoPort = "/dev/ttyACM0"
    gripperController = Motors(servoPort)
    gripperController.torquelimit(600) # used to be 600
    gripperController.speedlimit(100)
    try:
        ur.start()
        ur.gripper = gripperController
    except Exception as e:
        raise(e)
    else:
        print("UR5 Interface Established")
            
    homePose = ur.get_tcp_pose()
    pcheck = ur.getPose()
    print("Home Pose: ", homePose)
    print("Pose Check: ", pcheck)
    
    dX,dY,dZ = gt.get_world_frame(grasp_pose, ur)
    

    '''
    stopping point 2/15:
    this works so far
    take home pos
    add translation on the gripper frame to the goal x, y
    apply tmat to the goal
    move to goal (translates on x, y and orients at the same time)
    ## does this address the fact that orientation is not achievable in standstill?

    then: take orig Z (from original gripper frame): [0, 0, dZ]
    transform by tmat (the current orientation of the gripper)
    add transformed dZ to the goal position
    move
    works!
    '''
    goal1 = copy.deepcopy(homePose)
    goal1 = np.array(goal1)
    goal1[:3, 3] += [dX, dY, 0]
    # goal1.t[0] += dX
    # goal1.t[1] += dY
    goal1 = np.array(goal1) @ tmat
    # goal1 = np.array(goal1)
    ur.moveL(goal1)

    # todo: take currentPose and validate orientation?
    time.sleep(sleepRate)
    goal2 = goal1
    z_off = -0.00
    # posd = [0, 0, dZ - z_off] @ tmat[:3, :3]
    posd = [0, 0, dZ - z_off]
    goal2 = np.array(goal2)
    goal2[:3, 3] += posd
    # ur.moveL(goal2)

    # goal2.t[2] += dZ - 0.03
    # goal3 = np.array(goal2) @ tmat
    # ur.moveL(goal3)
    # time.sleep(sleepRate)

    '''
    end of working section
    '''
    
    currentPose = ur.get_tcp_pose()
    # desiredPose = np.matmul(np.array(currentPose), np.linalg.inv(tmat))
    # pos = np.array(currentPose.t)
    desiredPose = np.matmul(np.array(currentPose), tmat)
    # desiredPose[:3, 3] = pos
    # desiredPose = np.matmul(np.array(currentPose), tmat_full)
    print(f"currentPose: {currentPose}")
    print("Desired Pose: ", desiredPose)
    # ur.moveL(desiredPose)
    
    #gt.moveToObj(grasp_pose, ur) 
    

    print("Done moving to block")
    # ur.closeGripper(9) 
    time.sleep(sleepRate)
    # gt.moveBackFromObj(homePose, ur)
    # gt.grabStrat1(grasp_pose)
    ur.moveL(homePose)
    time.sleep(sleepRate)
    # ur.openGripper()
    ur.stop()
except Exception as e:
    gripperController.openGripper()
    gripperController.disconnect()
    ur.stop()
    raise(e)

Succeeded to open the port
Succeeded to change the baudrate
Moving speed of dxl ID: 1 set to 100 
Moving speed of dxl ID: 2 set to 100 
Moving speed of dxl ID: 1 set to 100 
Moving speed of dxl ID: 2 set to 100 
Succeeded to open the port
Succeeded to change the baudrate
Moving speed of dxl ID: 1 set to 100 
Moving speed of dxl ID: 2 set to 100 
UR5 Interface Established
Home Pose:  [[-0.997  0.081 -0.017 -0.088]
 [ 0.08   0.888 -0.452 -0.548]
 [-0.022 -0.452 -0.892  0.216]
 [ 0.     0.     0.     1.   ]]
Pose Check:    -0.9966    0.081    -0.01658  -0.08792   
   0.07973   0.8883   -0.4523   -0.5482    
  -0.02191  -0.452    -0.8917    0.2156    
   0         0         0         1         

[[-0.997  0.081 -0.017 -0.088]
 [ 0.08   0.888 -0.452 -0.548]
 [-0.022 -0.452 -0.892  0.216]
 [ 0.     0.     0.     1.   ]]
P_goal:
[-0.017 -0.044 -0.222]
currentPose: [[-0.997  0.081 -0.017 -0.105]
 [ 0.08   0.888 -0.452 -0.593]
 [-0.022 -0.452 -0.892  0.216]
 [ 0.     0.     0.     1.   ]]
Desir

RTDEControlInterface: RTDE control script is not running!
RTDEControlInterface: RTDE control script is not running!


In [5]:
# Full test
sleepRate = 1.5

mc = [0.018320640872559864, 0.05933741490882, 0.2980767279668537]
grasp_pose = mc
# grasp_pose = [mc[1], -mc[0], mc[2]]

try:
    robotIP = "192.168.0.6"
    con = rtde_control.RTDEControlInterface(robotIP)
    rec = rtde_receive.RTDEReceiveInterface(robotIP)
    ur = ur5.UR5_Interface()
    try:
        ur.c = con
        ur.r = rec
    except Exception as e:
        raise(e)
    else:
        print("UR5 Interface Established")
            
    # homePose = ur.getPose()
    # gt.moveToObj(grasp_pose, ur) 
    # print("Done moving to block")
    # ur.closeGripper(9) 
    # time.sleep(sleepRate)
    # gt.moveBackFromObj(homePose, ur)
    # ur.openGripper()
    # currentPose = ur.getPose()
    # desiredPose = np.matmul(np.array(currentPose), tmat)
    # print(f"currentPose: {currentPose}")
    # print("Desired Pose: ", desiredPose)
    ur.moveL(homePose)

    ur.c.disconnect()
    ur.r.disconnect()
except Exception as e:
    ur.c.disconnect()
    ur.r.disconnect()
    raise(e)

UR5 Interface Established


AttributeError: 'NoneType' object has no attribute 'moveL'